In [23]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [24]:
#import dataset
data = pd.read_csv("COMP30027_2021_Project2_datasets/recipe_train.csv")
testData = pd.read_csv("COMP30027_2021_Project2_datasets/recipe_test.csv")
data.head()

,name,n_steps,n_ingredients,steps,ingredients,duration_label
0,wi gal s chicken rice stove top dish made heal...,6,12,"['in saucepan , melt margarine and combine wit...","['margarine', 'olive oil', 'celery', 'onion', ...",2.0
1,irish pin oats,9,5,['melt 1 tbs butter in a small saucepan over m...,"['butter', 'pinhead oats', 'water', 'half-and-...",2.0
2,cheesy beef n biscuit casserole,15,10,"['brown ground beef , onion and green pepper',...","['ground beef', 'onion', 'green pepper', 'toma...",2.0
3,lemonade chicken oamc,10,8,"['brown chicken in oil', 'combine all ingredie...","['boneless skinless chicken', 'frozen lemonade...",2.0
4,graham and peanut butter bon bons,6,5,"['set almond bark aside', 'mix remaining ingre...","['graham cracker crumbs', 'crunchy peanut butt...",2.0


In [25]:
#Data Preprocessing - "[aa, bb]" -> "aa, bb"
data['steps'] = data['steps'].apply(eval).apply(' '.join)
data['ingredients'] = data['ingredients'].apply(eval).apply(' '.join)
testData['steps'] = testData['steps'].apply(eval).apply(' '.join)
testData['ingredients'] = testData['ingredients'].apply(eval).apply(' '.join)

In [26]:
#TfidfVectorizer - steps
vectorizer = TfidfVectorizer(stop_words="english").fit(data['steps'])
steps_vocab_tf = np.array(vectorizer.get_feature_names())
train_steps_tfidf = vectorizer.transform(data['steps'])
test_steps_tfidf = vectorizer.transform(testData['steps'])

In [27]:
y_train = data['duration_label']
def write_to_csv(predicted, file):
    print("write to csv...")
    df = pd.DataFrame({'id': np.arange(1, 10001),
                   'duration_label': predicted})
    df.to_csv(file, index=False)

In [28]:
#Linear SVC
svc = Pipeline([('kBest', SelectKBest(chi2, k=3500)), ('svc', LinearSVC())])
svc.fit(train_steps_tfidf, y_train)
scores_svc = cross_val_score(svc, train_steps_tfidf, y_train, cv=10)
predictedSVC = svc.predict(test_steps_tfidf)
print(np.mean(scores_svc))

0.784775


In [29]:
write_to_csv(predictedSVC, 'svc.csv')

write to csv...


In [30]:
#MNB
mnb = MultinomialNB()
mnb.fit(train_steps_tfidf, y_train)
scores_mnb = cross_val_score(mnb, train_steps_tfidf, y_train, cv=10)
predictedMNB = mnb.predict(test_steps_tfidf)
print(np.mean(scores_mnb))

0.699775


In [31]:
write_to_csv(predictedMNB, 'mnb.csv')

write to csv...


In [34]:
#KNN
neigh = KNeighborsClassifier(n_neighbors=40)
neigh.fit(train_steps_tfidf, y_train)
scores_knn = cross_val_score(neigh, train_steps_tfidf, y_train, cv=10)
predictedKNN = neigh.predict(test_steps_tfidf)
print(np.mean(scores_knn))

0.735125


In [35]:
write_to_csv(predicted, 'knn.csv')

write to csv...
